# Introduction to SQLite &
Selecting Sources from the Sloan Digital Sky Survey
========

#### Version 0.2

***
By AA Miller 2019 Mar 25  
Updated by G Hosseinzadeh 2025 Apr 9


This notebook provides a quick introduction to [`SQLite`](https://sqlite.org/index.html) a lightweight implementation of a Structured Query Language (SQL) database. One of the incredibly nice things about `SQLite` is the low overhead needed to set up a database (as you will see in a minute). We will take advantage of this low overhead to build a database later in the week.

At the most basic level - databases *store your bytes*, and later *return those bytes* (or a subset of them) when queried. 

They provide a highly efficient means for filtering your bytes (there are many different strategies that the user can employ). 

The backend for most databases is the **Structured Query Language** or SQL, which is a standard declarative language. 

There are many different libraries that implement SQL: MySQL, PostgreSQL, Greenplum, Microsoft SQL server, IBM DB2, Oracle Database, etc. 

The most basic implementation is [`SQLite`](https://www.sqlite.org) a self-contained, SQL database engine. We will discuss `SQLite` further later in the week, but in brief - it is a nice stand alone package that works really well for small problems (such as the example that we are about to encounter). Jupyter has a convenient interface to SQL that we will use here. Just load the SQL extension and connect to a new database. In the future, use the `%sql` (single line) or `%%sql` (entire cell) magic command to run SQL commands in your notebook.

In [1]:
%load_ext sql
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

In [2]:
%sql sqlite:///intro.db

And just like that - we have now created a new database `intro.db`, with which we can "store bytes" or later "retrieve bytes" once we have added some data to the database. 

*Aside* - note that unlike many SQL libraries, `SQLite` does not require a server and creates an actual database file on your hard drive. This improves portability, but also creates some downsides as well.

## Problem 1) Basic SQL Operations with SQLite

Now we need to create a table and insert some data. Recall that creating a table requires a specification of the table name, the columns in the table, and the data type for each column. Here's an example where I create a table to store info on my pets:


    %%sql
    create table PetInfo(  
                         Name text, 
                         Species text,
                         Age tinyint,
                         FavoriteFood text
                         )

**Problem 1a**

Create a new table in the database called `Students` with columns `Name`, `College`, and `Year`, where `Year` is the year in school.

In [7]:
%%sql 
create table Students(  
                     Name text, 
                     College text,
                     Year tinyint
                     )

 * sqlite:///intro.db
Done.


[]

Once a table is created, we can use the database to store bytes. If I were to populate my `PetInfo` table I would do the following:

    %%sql
    insert into PetInfo(Name, Species, Age, FavoriteFood) 
                        values ("Rocky", "Dog", 12, "Bo-Nana");
    insert into PetInfo(Name, Species, Age, FavoriteFood) 
                        values ("100 Emoji-Flames Emoji", "Red Panda", 2, "bamboo leaves");

*Note* - column names do not need to be explicitly specified, but for clarity this is always preferred.

**Problem 1b**

Insert data for yourself, and the two people sitting next to you into the database.

In [8]:
%%sql
insert into Students(Name, College, Year) 
                    values ("Noah", "UCSD", 3);
insert into Students(Name, College, Year) 
                    values ("Ishaan", "UCSD", 3);
insert into Students(Name, College, Year) 
                    values ("Lucas", "UCSD", 3);


 * sqlite:///intro.db
1 rows affected.
1 rows affected.
1 rows affected.


[]

Now that we have bytes in the database, we can retrieve those bytes with one (or several) queries. There are 3 basic building blocks to a query:

    SELECT...
    FROM...
    WHERE...
    
Where `SELECT` specifies the information we want to retrieve from the database, `FROM` specifies the tables being queried in the database, and `WHERE` specifies the conditions for the query. 

**Problem 1c**

Select the institutions for all students in the `Students` table who have been in school for more than 2 years.

In [9]:
%%sql select college
from Students
where year > 2;


 * sqlite:///intro.db
Done.


College
UCSD
UCSD
UCSD


## Problem 2) Complex Queries with SDSS

Above we looked at the most basic operations possible with a database (recall - databases are unnecessary, and possibly cumbersome, with small data sets). A typical database consists of many tables, and these tables may be joined together to unlock complex questions for the data. 

As a reminder on (some of) this functionality, we are now going to go through some problems using the SDSS database. The full [SDSS schema](http://skyserver.sdss.org/dr13/en/help/browser/browser.aspx) explains all of the tables, columns, views and functions for querying the database. We will keep things relatively simple in that regard.

In [10]:
from astroquery.sdss import SDSS

[`astroquery`](http://astroquery.readthedocs.io/en/latest/) enables seemless connections to the SDSS database via the Python shell.

**Problem 2a**

Select 20 random sources from the [`PhotoObjAll`](https://skyserver.sdss.org/dr13/en/help/docs/tabledesc.aspx?name=PhotoObjAll) table and return all columns in the table. 

*Hint* - while this would normally be accomplished by starting the query `select limit 20 ...`, SDSS CasJobs uses Microsoft's SQL Server, which adopts `select top 20 ...` to accomplish an identical result.

In [14]:
SDSS.query_sql( "SELECT TOP 20 * FROM PhotoObjAll")

objID,skyVersion,run,rerun,camcol,field,obj,mode,nChild,type,clean,probPSF,insideMask,flags,rowc,rowcErr,colc,colcErr,rowv,rowvErr,colv,colvErr,rowc_u,rowc_g,rowc_r,rowc_i,rowc_z,rowcErr_u,rowcErr_g,rowcErr_r,rowcErr_i,rowcErr_z,colc_u,colc_g,colc_r,colc_i,colc_z,colcErr_u,colcErr_g,colcErr_r,colcErr_i,colcErr_z,sky_u,sky_g,sky_r,sky_i,sky_z,skyIvar_u,skyIvar_g,skyIvar_r,skyIvar_i,skyIvar_z,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,psfMagErr_u,psfMagErr_g,psfMagErr_r,psfMagErr_i,psfMagErr_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,fiberMag_z,fiberMagErr_u,fiberMagErr_g,fiberMagErr_r,fiberMagErr_i,fiberMagErr_z,fiber2Mag_u,fiber2Mag_g,fiber2Mag_r,fiber2Mag_i,fiber2Mag_z,fiber2MagErr_u,fiber2MagErr_g,fiber2MagErr_r,fiber2MagErr_i,fiber2MagErr_z,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,petroMagErr_u,petroMagErr_g,petroMagErr_r,petroMagErr_i,petroMagErr_z,psfFlux_u,psfFlux_g,psfFlux_r,psfFlux_i,psfFlux_z,psfFluxIvar_u,psfFluxIvar_g,psfFluxIvar_r,psfFluxIvar_i,psfFluxIvar_z,fiberFlux_u,fiberFlux_g,fiberFlux_r,fiberFlux_i,fiberFlux_z,fiberFluxIvar_u,fiberFluxIvar_g,fiberFluxIvar_r,fiberFluxIvar_i,fiberFluxIvar_z,fiber2Flux_u,fiber2Flux_g,fiber2Flux_r,fiber2Flux_i,fiber2Flux_z,fiber2FluxIvar_u,fiber2FluxIvar_g,fiber2FluxIvar_r,fiber2FluxIvar_i,fiber2FluxIvar_z,petroFlux_u,petroFlux_g,petroFlux_r,petroFlux_i,petroFlux_z,petroFluxIvar_u,petroFluxIvar_g,petroFluxIvar_r,petroFluxIvar_i,petroFluxIvar_z,petroRad_u,petroRad_g,petroRad_r,petroRad_i,petroRad_z,petroRadErr_u,petroRadErr_g,petroRadErr_r,petroRadErr_i,petroRadErr_z,petroR50_u,petroR50_g,petroR50_r,petroR50_i,petroR50_z,petroR50Err_u,petroR50Err_g,petroR50Err_r,petroR50Err_i,petroR50Err_z,petroR90_u,petroR90_g,petroR90_r,petroR90_i,petroR90_z,petroR90Err_u,petroR90Err_g,petroR90Err_r,petroR90Err_i,petroR90Err_z,q_u,q_g,q_r,q_i,q_z,qErr_u,qErr_g,qErr_r,qErr_i,qErr_z,u_u,u_g,u_r,u_i,u_z,uErr_u,uErr_g,uErr_r,uErr_i,uErr_z,mE1_u,mE1_g,mE1_r,mE1_i,mE1_z,mE2_u,mE2_g,mE2_r,mE2_i,mE2_z,mE1E1Err_u,mE1E1Err_g,mE1E1Err_r,mE1E1Err_i,mE1E1Err_z,mE1E2Err_u,mE1E2Err_g,mE1E2Err_r,mE1E2Err_i,mE1E2Err_z,mE2E2Err_u,mE2E2Err_g,mE2E2Err_r,mE2E2Err_i,mE2E2Err_z,mRrCc_u,mRrCc_g,mRrCc_r,mRrCc_i,mRrCc_z,mRrCcErr_u,mRrCcErr_g,mRrCcErr_r,mRrCcErr_i,mRrCcErr_z,mCr4_u,mCr4_g,mCr4_r,mCr4_i,mCr4_z,mE1PSF_u,mE1PSF_g,mE1PSF_r,mE1PSF_i,mE1PSF_z,mE2PSF_u,mE2PSF_g,mE2PSF_r,mE2PSF_i,mE2PSF_z,mRrCcPSF_u,mRrCcPSF_g,mRrCcPSF_r,mRrCcPSF_i,mRrCcPSF_z,mCr4PSF_u,mCr4PSF_g,mCr4PSF_r,mCr4PSF_i,mCr4PSF_z,deVRad_u,deVRad_g,deVRad_r,deVRad_i,deVRad_z,deVRadErr_u,deVRadErr_g,deVRadErr_r,deVRadErr_i,deVRadErr_z,deVAB_u,deVAB_g,deVAB_r,deVAB_i,deVAB_z,deVABErr_u,deVABErr_g,deVABErr_r,deVABErr_i,deVABErr_z,deVPhi_u,deVPhi_g,deVPhi_r,deVPhi_i,deVPhi_z,deVMag_u,deVMag_g,deVMag_r,deVMag_i,deVMag_z,deVMagErr_u,deVMagErr_g,deVMagErr_r,deVMagErr_i,deVMagErr_z,deVFlux_u,deVFlux_g,deVFlux_r,deVFlux_i,deVFlux_z,deVFluxIvar_u,deVFluxIvar_g,deVFluxIvar_r,deVFluxIvar_i,deVFluxIvar_z,expRad_u,expRad_g,expRad_r,expRad_i,expRad_z,expRadErr_u,expRadErr_g,expRadErr_r,expRadErr_i,expRadErr_z,expAB_u,expAB_g,expAB_r,expAB_i,expAB_z,expABErr_u,expABErr_g,expABErr_r,expABErr_i,expABErr_z,expPhi_u,expPhi_g,expPhi_r,expPhi_i,expPhi_z,expMag_u,expMag_g,expMag_r,expMag_i,expMag_z,expMagErr_u,expMagErr_g,expMagErr_r,expMagErr_i,expMagErr_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z,modelMagErr_u,modelMagErr_g,modelMagErr_r,modelMagErr_i,modelMagErr_z,cModelMag_u,cModelMag_g,cModelMag_r,cModelMag_i,cModelMag_z,cModelMagErr_u,cModelMagErr_g,cModelMagErr_r,cModelMagErr_i,cModelMagErr_z,expFlux_u,expFlux_g,expFlux_r,expFlux_i,expFlux_z,expFluxIvar_u,expFluxIvar_g,expFluxIvar_r,expFluxIvar_i,expFluxIvar_z,modelFlux_u,modelFlux_g,modelFlux_r,modelFlux_i,modelFlux_z,modelFluxIvar_u,modelFluxIvar_g,modelFluxIvar_r,modelFluxIvar_i,modelFluxIvar_z,cModelFlux_u,cModelFlux_g,cModelFlux_r,cModelFlux_i,cModelFlux_z,cModelFluxIvar_u,cModelFluxIvar_g,cModelFluxIvar_r,cModelFluxIvar_i,cModelFluxIvar_z,aperFlux7_u,aperFlux7_g,aperFlux7_r,aperF

That's more columns than we will likely ever need. Instead, let's focus on `objID`, a unique identifier, `cModelMag_u`, `cModelMag_g`, `cModelMag_r`, `cModelMag_i`, and `cModelMag_z`, the source magnitude in $u', g', r', i', z'$, respectively.

We will now introduce the concept of joining two tables. 

The most common operation is known as an `inner join` (which is often referred to as just `join`). An `inner join` returns records that have matching sources in both tables in the join. 

Less, but nevertheless still powerful, is the `outer join`. An outer join returns *all* records in either table, with `NULL` values for columns in a table in which the record does not exist.

Specialized versions of the `outer join` include the `left join` and `right join`, whereby *all* records in either the left or right table, respectively, are returned along with their counterparts. 

**Problem 2b**

Select `objid` and $u'g'r'i'z'$ from `PhotoObjAll` and the corresponding `class` from [`specObjAll`](https://skyserver.sdss.org/dr13/en/help/docs/tabledesc.aspx?name=SpecObjAll) for 20 random sources. 

There are multiple columns you could use to join the tables, in this case match `objid` to `bestobjid` from `specObjAll` and use an `inner join`.

In [20]:
SDSS.query_sql( "SELECT TOP 20 objid, psfMag_u, psfMag_g, psfMag_r, psfMag_i, psfMag_z, class FROM PhotoObjAll INNER JOIN specObjAll ON objid = bestobjid")

objid,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,class
uint64,float64,float64,float64,float64,float64,str6
1237666309095751941,25.29274,23.9807,21.82987,20.77349,20.15467,GALAXY
1237669516904956169,21.59819,20.47092,19.87875,19.61314,19.47023,STAR
1237669516904956169,21.59819,20.47092,19.87875,19.61314,19.47023,STAR
1237663233379074508,21.75525,20.85893,20.55289,20.52644,20.4651,STAR
1237678881020641645,26.07794,22.80113,21.57976,20.26642,19.62477,STAR
1237663305845506792,26.08331,23.12045,21.90256,20.82006,20.03485,GALAXY
1237680243597902184,22.92114,23.16882,21.75551,20.82732,20.26526,GALAXY
1237679319089349467,26.07469,23.86481,22.05336,20.85726,20.2834,GALAXY
1237666184574271705,21.94773,19.62906,18.44153,18.05969,17.74265,GALAXY


**Problem 2c**

Perform an identical query to the one above, but this time use a `left outer join` (or `left join`).

How do your results compare to the previous query?

In [21]:
SDSS.query_sql( "SELECT TOP 20 objid, psfMag_u, psfMag_g, psfMag_r, psfMag_i, psfMag_z, class FROM PhotoObjAll LEFT JOIN specObjAll ON objid = bestobjid")

objid,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,class
uint64,float64,float64,float64,float64,float64,int64
1237666300533604922,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,--
1237663232842859204,24.15211,22.87897,22.70212,22.15132,23.70784,--
1237663783660552229,24.0344,23.55712,22.58595,22.05489,21.1335,--
1237660025040601553,22.83112,22.7966,21.78695,21.46582,21.50661,--
1237672837454299551,24.59541,22.47992,21.3644,20.68542,20.21314,--
1237663203840426399,24.69642,22.99697,21.97336,21.6932,20.81135,--
1237663203840426397,24.88073,22.93774,21.97654,21.68166,20.78332,--
1237679435055366468,23.03828,22.15162,21.71507,21.69623,20.9957,--
1237679435055366466,23.0383,22.15164,21.71507,21.14418,20.30687,--


**Problem 2d**

This time use a `right outer join` (or `right join`).

How do your results compare to the previous query?

In [24]:
SDSS.query_sql( """SELECT TOP 20 objid, psfMag_u, psfMag_g, psfMag_r, psfMag_i, psfMag_z, class 
FROM PhotoObjAll RIGHT JOIN specObjAll ON objid = bestobjid""")

objid,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,class
uint64,float64,float64,float64,float64,float64,str6
--,--,--,--,--,--,QSO
1237666309095751941,25.29274,23.9807,21.82987,20.77349,20.15467,GALAXY
1237669516904956169,21.59819,20.47092,19.87875,19.61314,19.47023,STAR
1237669516904956169,21.59819,20.47092,19.87875,19.61314,19.47023,STAR
1237663233379074508,21.75525,20.85893,20.55289,20.52644,20.4651,STAR
1237678881020641645,26.07794,22.80113,21.57976,20.26642,19.62477,STAR
1237663305845506792,26.08331,23.12045,21.90256,20.82006,20.03485,GALAXY
1237680243597902184,22.92114,23.16882,21.75551,20.82732,20.26526,GALAXY
1237679319089349467,26.07469,23.86481,22.05336,20.85726,20.2834,GALAXY


## Challenge Problem

To close the notebook we will perform a nested query. In brief, the idea is to join the results of one query with a separate query.

Here, we are going to attempt to identify bright AGN that don't have SDSS spectra. To do so we will need the `photoObjAll` table, the `specObjAll` table, and the `rosat` table, which includes all cross matches between SDSS sources and X-ray sources detected by the [Rosat satellite](https://heasarc.gsfc.nasa.gov/docs/rosat/rosat3.html).

Create a nested query that selects all *Rosat* sources that don't have SDSS spectra with `cModelFlux_u + cModelFlux_g + cModelFlux_r + cModelFlux_i + cModelFlux_z > 10000` (this flux contraint ensures the source is bright without making any cuts on color) and `type = 3`, this last constraint means the source is extended in SDSS images. 

*Hint* - you may run into timeout issues in which case you should run the query on CasJobs.

In [30]:
SDSS.query_sql("""SELECT p.cModelFlux_u, p.cModelFlux_g, p.cModelFlux_r, p.cModelFlux_i, p.cModelFlux_z, r.objid, p.objid
    FROM rosat 
    Left JOIN photoObjAll ON  r.objid = p.objid
    WHERE (p.cModelFlux_u + p.cModelFlux_g + p.cModelFlux_r + p.cModelFlux_i + p.cModelFlux_z) > 10000
    AND type = 3
""")

InconsistentTableError: Number of header columns (1) inconsistent with data columns in data line 2